In [2]:
"""
Chapter 10. Network Models. Shortest Path.
Anderson, David R., Dennis J. Sweeney,
Thomas A. Williams,
Jeffrey D. Camm y Kipp Martin
Quantitative Methods for Business, 11a. Ed
"""
from itertools import product
import math

import pulp as pl


In [3]:

nodes = range(6)

# adjacency_matrix
a = [
    [0, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 0, 1],
    [0, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 1, 1],
    [0, 0, 1, 1, 0, 1],
    [0, 0, 0, 0, 0, 0],
    ]

costs = [
   [1_000, 25, 20,1_000,1_000,1_000],
   [1_000,1_000, 3, 5,1_000, 14],
   [1_000, 3,1_000,1_000, 6,1_000],
   [1_000, 5,1_000,1_000, 4, 4],
   [1_000,1_000, 6, 4,1_000, 7],
   [1_000,1_000,1_000,1_000,1_000,1_000],
    ]

load = {
    0: 1, 
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: -1,
}

In [4]:

costs = pl.makeDict((nodes, nodes), costs, 1_000)


# problem
problem = pl.LpProblem('shortest', pl.LpMinimize)


# variables
path = pl.LpVariable.dicts('path', (nodes, nodes), 0, None, pl.LpContinuous)

# objective function
problem += pl.lpSum(costs[i][j] * path[i][j] for j in nodes for i in nodes), 'sum_time_path'


# Transhipment Constraints
for k in nodes:
    problem += pl.lpSum(a[k][out] * path[k][out] for out in nodes) - pl.lpSum(a[enter][k] * path[enter][k] for enter in nodes) == load[k], f'path_to_node_{k}'


In [5]:

solver = pl.get_solver('GUROBI') 
problem.solve(solver)

# The status of the solution is printed to the screen
print("Status:", pl.LpStatus[problem.status])


# Each of the variables is printed with it's resolved optimum value
for v in problem.variables():
    if v.varValue > 1e-10:
        print(f'{v.name}: {v.varValue:,.2f}')

print(f'Total Miles = {pl.value(problem.objective):,.2f}')


Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 36 columns and 26 nonzeros
Model fingerprint: 0xb776b652
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 24 columns
Presolve time: 0.01s
Presolved: 5 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+01   2.000000e+00   0.000000e+00      0s
       4    3.2000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.200000000e+01
Gurobi status= 2
Status: Optimal
path_0_2: 1.00
path_1_3: 1.00
path_2_1: 1.00
path_3_5: 1.00
Total Miles = 32.00
